In [1]:
from urllib import *
from html import unescape
import requests
import json 
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

In [23]:
data0 = {"vPstrCategory": "TOT",
         "vPstrKeyWord": requests.compat.quote("박보영"),
         "vPplace": "top"
        }

In [24]:
data1 = {
         "vPstrKeyWord": "박보영"
        
        }

In [25]:
requests.compat.quote("박보영")

'%EB%B0%95%EB%B3%B4%EC%98%81'

In [26]:
data2 = {"vPstrCategory": "TOT",
         "vPstrKeyWord": "may",
         "vPplace": "top"
        }

In [2]:
import requests
import time

def download(method, url, 
             param = None, data = None, 
            timeout = 1,maxretries = 3):
    
    #headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
    headers = None
    
    try:
        resp = requests.request(method, url, params= param, data=data, headers=headers)
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0: 
            time.sleep(timeout) # param에따라 몇초 기다릴지 결정 할 수가 있다.
            print(maxretries) # 재귀적으로 자기 자신을 부르게 코드를 짜면 된다.
            resp = download(method,
                            url, param = param, data=data,
                            timeout=timeout,
                            maxretries = maxretries - 1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [28]:
html = download("post", "http://www.kyobobook.co.kr/search/SearchCommonMain.jsp",data1)

In [30]:

#print(html.text)


In [ ]:
#####################      세션 쿠키

In [31]:
## http://pythonscraping.com/pages/cookies/login.html
# 로그인 했을때 쿠키 심음

#loggedin  을 트루라고 해놓고 
#id, paswword 없어도 해시값이나 이런거 ttl 넣어서 심어놓음 30일정도지나면 로그인 풀려있고
#login 정보알아야 뒷단에 데이터베이스  볼수있따

##일반적으로 form 태그안에  어떤 작동해야 폼이 바뀐다 ( 자바스크립트는 다르게 만들수도)

##네이버나 다음에 폼태그 어딨는지 봐보기


## <form> 어떤 동작에 어떤 intent

## url 조인 url 뒤에 다붙어주는거

##url조인 request, urllib에 있다.

In [34]:
requests.compat.urlparse( "http://pythonscraping.com/pages/cookies/login.html?id=1234")

ParseResult(scheme='http', netloc='pythonscraping.com', path='/pages/cookies/login.html', params='', query='id=1234', fragment='')

In [ ]:
## 파라메터도 다쪼개준다.
## 쪼갠거 기반으로 urljoin 해준다

In [35]:
requests.compat.urljoin("http://pythonscraping.com/pages/cookies/login.html?id=1234", "process.php")

'http://pythonscraping.com/pages/cookies/process.php'

In [5]:
from bs4 import BeautifulSoup

In [11]:
url = "http://pythonscraping.com/pages/cookies/login"   ## http 안붙이면 안된다. http https 스키마 없다
url = requests.compat.urljoin(url, "welcome.php")
data = {
    "username" : "1234",
    "password" : "password"
}

html = download("post", url, data=data)
print(html.text)
dom = BeautifulSoup(html.text, "lxml")
print("============")
dom


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


<html><body><h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br/><a href="login.html">Log in here</a></body></html>

In [ ]:
## 쿠키나 세션값 안받아옴 logged in = 1 로되야 로그인 처리되는데
##

In [ ]:
## download 함수 ~~ .py로 떨궈놓기 import하게

In [46]:
## 세션, 쿠키 받아옴


In [47]:
session = requests.Session()

In [48]:
   ##쿠키가 생성이됨 세션, 쿠키 로그인한채로 들고 이동 가능

In [40]:
html  = session.post(url, data)   ## 최초 접속했을때, 쿠키는 생성했지만 반영이 안되서 로그인이 안되있다.
html.text

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [41]:
html  = session.post(url)  ## 동일하게 접속했을때 쿠키가 살아있어서 로그인이 됬다고 나옴
html.text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

In [42]:
session.cookies  ## loggedin value 가 1로설정

<RequestsCookieJar[Cookie(version=0, name='loggedin', value='1', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='username', value='1234', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [ ]:
## 대형 포털은 로그인 처리 여러번 됨
## 단일 페이지에서 로그인처리할때만됨 소규모 사이트 but 우리나라 커뮤니티 다뚤린다
## 로그인되면 db에있는거 다 긁어올수 있다.
## 폼태그, 액션, 인풋태그, 벨류 어떻게심는지, 쿠키생성됬는지, 로그인된 결과 받아올건지
## 더미데이터 수천개 풀고 게시글쓰고 추천누르고 좋아요누르고 특정게시물 좋아요 ㅈㄴ누르고 이런거 가능

In [49]:
as

NameError: name 'asdasd' is not defined